In [66]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.models import load_model # type: ignore
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB0 # type: ignore
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization # type: ignore
from tensorflow.keras.models import Model # type: ignore
import shutil
import random

In [67]:
TRAIN_DIR = '/Users/michelangelozampieri/Desktop/plastics_CNN/seven_plastics'
TEST_DIR = '/Users/michelangelozampieri/Desktop/plastics_CNN/seven_plastics_test'
IMG_SIZE = (244, 244)
BATCH_SIZE = 32
NUM_CLASSES = 8
EPOCHS_INITIAL = 5
EPOCHS_FINE_TUNE = 10

In [68]:
test_split_ratio = .3

# Create the test directory if it doesn't exist
os.makedirs(TEST_DIR, exist_ok=True)

# Iterate through each category in the training directory
for category in os.listdir(TRAIN_DIR):
    category_path = os.path.join(TRAIN_DIR, category)
    if os.path.isdir(category_path):
        # Create a corresponding category directory in the test directory
        test_category_path = os.path.join(TEST_DIR, category)
        os.makedirs(test_category_path, exist_ok=True)

        # Get all image files in the category
        images = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        
        # Randomly select a subset of images for testing
        num_test_images = int(len(images) * test_split_ratio)
        test_images = random.sample(images, num_test_images)

        # Move the selected images to the test directory
        for image in test_images:
            src_path = os.path.join(category_path, image)
            dest_path = os.path.join(test_category_path, image)
            shutil.move(src_path, dest_path)

print("Dataset split completed!")

Dataset split completed!


In [69]:
# Get a list of all categories (subdirectories) in TRAIN_DIR
categories = [d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))]

# Remove hidden files like .DS_Store
categories = [c for c in categories if not c.startswith(".")]

# Get a list of all categories (subdirectories) in TRAIN_DIR
categories = [entry.name for entry in os.scandir(TRAIN_DIR) if entry.is_dir()]

print(categories)

['4_low_density_polyethylene_PE-LD', '8_no_plastic', '3_polyvinylchloride_PVC', '1_polyethylene_PET', '6_polystyrene_PS', '5_polypropylene_PP', '7_other_resins', '2_high_density_polyethylene_PE-HD']


In [70]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=40,  # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20%
    height_shift_range=0.2,  # Randomly shift images vertically by 20%
    shear_range=0.2,  # Randomly apply shearing transformations
    zoom_range=0.2,  # Randomly zoom in/out on images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in missing pixels after transformations
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [71]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",  # Ensure labels are inferred from subdirectories
    label_mode="categorical",  # Use categorical labels for multi-class classification
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    labels="inferred",  # Ensure labels are inferred from subdirectories
    label_mode="categorical",  # Use categorical labels for validation
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 48 files belonging to 8 classes.


Found 294 files belonging to 8 classes.


In [72]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(244, 244, 3))
base_model.trainable = False  # Initially freeze the base model

# Custom classification head
x = GlobalAveragePooling2D()(base_model.output)  # Reduce feature maps to a single vector
x = Dense(64, activation="relu")(x)  # Fewer neurons for simplicity
x = BatchNormalization()(x)  # Optional: Stabilize training
output_layer = Dense(NUM_CLASSES, activation="softmax")(x)  # Output layer

# Create model
model = Model(inputs=base_model.input, outputs=output_layer)

In [73]:
# Step 3: Compile and Train (Feature Extraction Phase)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train for a few epochs with frozen base model

history = model.fit(
    train_dataset,
    epochs=EPOCHS_INITIAL
)

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.1493 - loss: 2.5234
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 255ms/step - accuracy: 0.5764 - loss: 1.2512
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 238ms/step - accuracy: 0.7778 - loss: 0.6973
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - accuracy: 0.8889 - loss: 0.5522
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 303ms/step - accuracy: 0.9375 - loss: 0.3675


In [74]:
model.save("plastics_model_v1.h5")

In [78]:
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 5s/step - accuracy: 0.3450 - loss: 1.7747
Validation accuracy: 0.35374149680137634
